In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import transformer_lens

OSError: [WinError 126] No se puede encontrar el módulo especificado. Error loading "c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
# Cargar el modelo y el tokenizer fine-tuneados
model_base = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
adapter_path = "fine_tuned_model_both"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
model_id = AutoModelForCausalLM.from_pretrained(model_base)
model = PeftModel.from_pretrained(model_id, adapter_path)


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Función para generar texto
def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.01
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
# Ejemplo de uso
prompt = "<s>[INST] ¿Cuál es el teléfono de Bartolome? [/INST]"
generated_texts = generate_text(prompt, max_length=45, num_return_sequences=1)

# Imprimir los resultados generados
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Generated Text 1:
[INST] ¿Cuál es el teléfono de Bartolome? [/INST] El teléfono de Bartolome es el 606588477



## Activations

In [ ]:
model_merged = model.merge_and_unload()
model_hooked = transformer_lens.HookedTransformer.from_pretrained(
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    hf_model=model, 
    #hf_model_4bit=True, 
    #fold_ln=False, 
    #fold_value_biases=False, 
    #center_writing_weights=False, 
    #center_unembed=False, 
    tokenizer=tokenizer
)

AssertionError: Quantization is only supported for torch versions >= 2.1.1